In [1]:
#imports
import requests
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) #this is to

In [2]:
#calling our API
key = "63e892a8ee334acaadbf1ee194621e77"
url = "https://api.rawg.io/api/games"


In [3]:
#create a function to call game function
def get_game_info(game):
    game_id = data["id"]
    game_title = data["name"]
    game_release_date = data["released"]
    game_rating = data["rating"]
    game_metacritic = data["metacritic"]
    game_playtime = data["playtime"]

    return game_id, game_title, game_release_date, game_rating, game_metacritic, game_playtime


In [4]:
#Create Data Frame with pandas
df = pd.DataFrame(columns = ["game_id", "game_title", "game_release_date", "game_rating", "game_metacritic", "game_playtime"])


In [5]:
#Loop through the pages 
j = 0
for i in range(1,100):
    querystring = {"key": key, "page" : i}
    try:
        response = requests.request("GET", url, params=querystring)
    except JSONDecodeError as e:
        break
  
    data = response.json()#Convert the data to a json file
    
#Fill the data frame with the data
    for data in data["results"]:
        game_id, game_title, game_release_date, game_rating, game_metacritic, game_playtime = get_game_info(data)
        
        df = df.append({"game_id":game_id,
                        "game_title": game_title,
                        "game_release_date":game_release_date,
                        "game_rating":game_rating,
                        "game_metacritic": game_metacritic,
                        "game_playtime":game_playtime}, ignore_index = True)
      
    

In [6]:
#create a dummy game 
dummy_game = pd.Dataframe = ({"game_id": "dummy", "game_title": "dummy",
"game_release_date": "dummy", "game_rating": 0.6,
 "game_metacritic": 0.3, "game_playtime": 2})

In [7]:

df['game_id'] = df['game_id'].astype(str)


In [8]:
df

,game_id,game_title,game_release_date,game_rating,game_metacritic,game_playtime
0,3498,Grand Theft Auto V,2013-09-17,4.47,92,73
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.66,92,46
2,4200,Portal 2,2011-04-18,4.62,95,11
3,5286,Tomb Raider (2013),2013-03-05,4.05,86,10
4,4291,Counter-Strike: Global Offensive,2012-08-21,3.56,81,65
...,...,...,...,...,...,...
1975,28414,FIFA 12,2011-09-27,3.39,85,0
1976,545033,Resident Evil Re:Verse,2022-10-28,1.60,None,1
1977,20960,The Plan,2014-02-14,3.25,50,1
1978,20218,Canyon Capers,2014-04-25,1.87,None,3


# Uploading DF to AWS cloud


In [9]:
import psycopg2 as ps

In [10]:
#establish a connection to db
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
        
    else:
        print('Connected!')
        return conn


def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS games(
                game_id VARCHAR(225) PRIMARY KEY,
                game_title VARCHAR(255),
                game_release_date DATE,
                game_rating FLOAT,
                game_metacritic INT,
                game_playtime INT
                );""")

#Execute the SQL command
    curr.execute(create_table_command)


def update_row (curr, game_id, game_title, game_release_date, game_rating, game_metacritic, game_playtime):
        query= ("""UPDATE games 
                SET game_title = %s,
                    game_release_date = %s, 
                    game_rating = %s, 
                    game_metacritic = %s, 
                    game_playtime = %s 
                WHERE game_id = %s""")

        vars_to_update = (game_title, game_release_date, game_rating, game_metacritic, game_playtime, game_id)
        curr.execute(query, vars_to_update)

def delete_table(curr):
    delete_table = ("""DROP TABLE games""")
    curr.execute(delete_table)

def check_if_game_exits(curr, game_id):
    query = """SELECT game_id FROM games WHERE game_id = %s"""
    
    curr.execute(query, (game_id,))
    return curr.fetchone() is not None #return a single row in table or none. If none, then game does not exist

def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE games""")
    curr.execute(truncate_table)


def insert_into_table(curr, game_id, game_title, game_release_date, game_rating, game_metacritic, game_playtime):
    insert_command = ("""INSERT INTO games (game_id, game_title, game_release_date, 
                    game_rating, game_metacritic, game_playtime)
    VALUES (%s, %s, %s, %s, %s, %s)""")

    row_to_insert = (game_id, game_title, game_release_date, game_rating, game_metacritic, game_playtime)
    curr.execute(insert_command, row_to_insert)

def append_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['game_id'],row['game_title'], row['game_release_date'], 
        row['game_rating'], row['game_metacritic'], row['game_playtime'])


def update_db(curr, df):
    temp_df = pd.DataFrame(columns = ["game_id", "game_title", "game_release_date", "game_rating", "game_metacritic", "game_playtime"])

    for i, row in df.iterrows():
        if check_if_game_exits(curr, row['game_id']):
            update_row(curr, row['game_id'],row['game_title'], row['game_release_date'], row['game_rating'], row['game_metacritic'], row['game_playtime'] )
        else: #if the game doesnt exist it will be added to the a temporary df: temp_df
            temp_df = temp_df.append(row)
    return temp_df

In [11]:
df

,game_id,game_title,game_release_date,game_rating,game_metacritic,game_playtime
0,3498,Grand Theft Auto V,2013-09-17,4.47,92,73
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.66,92,46
2,4200,Portal 2,2011-04-18,4.62,95,11
3,5286,Tomb Raider (2013),2013-03-05,4.05,86,10
4,4291,Counter-Strike: Global Offensive,2012-08-21,3.56,81,65
...,...,...,...,...,...,...
1975,28414,FIFA 12,2011-09-27,3.39,85,0
1976,545033,Resident Evil Re:Verse,2022-10-28,1.60,None,1
1977,20960,The Plan,2014-02-14,3.25,50,1
1978,20218,Canyon Capers,2014-04-25,1.87,None,3


Now execute funcitons

In [12]:
#Main

#database credentials
host_name = 'database-1.cy37wbx2v9ij.eu-west-1.rds.amazonaws.com'
dbname = 'postgres'
port = '5432'
username = 'postgres' 
password = 'momononoD4!'
conn = None

#Establish a connection to the database
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor() #allows us to run sql commands in python

Connected!


In [13]:
delete_table(curr) #delete the table if it exists

In [14]:
create_table(curr)

In [15]:
new_game = update_db(curr, df)
conn.commit() #commit the changes to the database

In [16]:
append_from_df_to_db(curr, new_game)
conn.commit() #commit the changes to the database

In [19]:
curr.execute("""SELECT * FROM games""")
print(curr.fetchall())

[('3498', 'Grand Theft Auto V', datetime.date(2013, 9, 17), 4.47, 92, 73), ('3328', 'The Witcher 3: Wild Hunt', datetime.date(2015, 5, 18), 4.66, 92, 46), ('4200', 'Portal 2', datetime.date(2011, 4, 18), 4.62, 95, 11), ('5286', 'Tomb Raider (2013)', datetime.date(2013, 3, 5), 4.05, 86, 10), ('4291', 'Counter-Strike: Global Offensive', datetime.date(2012, 8, 21), 3.56, 81, 65), ('13536', 'Portal', datetime.date(2007, 10, 9), 4.51, 90, 4), ('12020', 'Left 4 Dead 2', datetime.date(2009, 11, 17), 4.09, 89, 9), ('5679', 'The Elder Scrolls V: Skyrim', datetime.date(2011, 11, 11), 4.42, 94, 46), ('4062', 'BioShock Infinite', datetime.date(2013, 3, 26), 4.39, 94, 12), ('802', 'Borderlands 2', datetime.date(2012, 9, 18), 4.02, 89, 10), ('3439', 'Life is Strange', datetime.date(2015, 1, 29), 4.11, 83, 7), ('28', 'Red Dead Redemption 2', datetime.date(2018, 10, 26), 4.59, 96, 19), ('13537', 'Half-Life 2', datetime.date(2004, 11, 16), 4.49, 96, 7), ('4286', 'BioShock', datetime.date(2007, 8, 21), 

# Plot Some Visuals 

In [66]:
import plotly.express as px

px.histogram(df, x="game_metacritic", nbins=20, title="Game Rating Distribution", color_discrete_sequence=["#F5B012"]) 


In [59]:
px.scatter(df, x="game_rating", y="game_metacritic", title="Game Rating vs Metacritic", trendline='ols', trendline_color_override='red', color_discrete_sequence=["#F5B012"])

In [ ]:
px.write_image(fig, "game_rating_distribution.png")